In [18]:
import pandas as pd
import numpy as np
#data visualization
import matplotlib.pyplot as plt
import librosa

import time

from scipy.stats import kurtosis
from scipy.stats import skew

from sklearn.model_selection import train_test_split

pd.options.display.precision = 10

## Umgebungsvariablen

In [2]:
feature_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/earthquakeFeatures_1553342293.3349762_.csv'''
cat_boost_model = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/catboost.dump'''

# Feature laden

In [3]:
train_data = pd.read_csv(feature_data_path, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [8]:
train_data.iloc[:,1:95].head()

,mean,std,kurt,skew,min,max,1%q,5%q,25%q,75%q,...,spec_bw_kurt,spec_bw_skew,spec_bw_min,spec_bw_max,spec_bw_1%q,spec_bw_5%q,spec_bw_25%q,spec_bw_75%q,spec_bw_95%q,spec_bw_99%q
0,4.8841133333,5.1011061306,33.6624812935,-0.0240611666,-98.0,104.0,-8.0,-2.0,3.0,7.0,...,12.7760446198,-2.9176274163,381349.1865900679,593848.0949585186,464802.9437660335,513229.8197951605,557429.6937546992,578910.9580815419,586597.8954874629,589247.3423812583
1,4.8571266667,4.3240072299,8.4650571655,0.0587541958,-56.0,52.0,-7.0,-1.0,3.0,7.0,...,3.0236650767,-1.6920770163,473999.4882262739,593584.9494675705,496767.4351236603,522165.6770328701,558931.0723155735,579312.3933050043,586314.7130109678,588315.3456784203
2,4.8376266667,5.3342163992,109.4686695306,0.3999465498,-154.0,181.0,-8.0,-2.0,3.0,7.0,...,23.1760591584,-3.9070500266,334862.6694803852,593503.1188252728,457056.1477548956,516048.5313389048,558020.4346092035,579521.9126454300,586696.8275960149,589603.5340530913
3,4.8111600000,5.3222446671,110.4614763269,0.4131461554,-154.0,181.0,-8.0,-2.0,2.0,7.0,...,23.7946792295,-3.9943724011,332568.2442054225,593958.4716608603,445689.0151435903,516651.7592641455,558113.8118867078,578970.1809539023,585994.1793593073,588331.0470630293
4,4.7928533333,5.4182032051,103.2711186532,0.3964407128,-154.0,181.0,-8.0,-2.0,2.0,7.0,...,21.3302543901,-3.7569395880,332795.9142868448,593107.2663205061,453205.2527339894,512382.8547605443,558646.1750177351,579336.2976062959,586284.5423266705,589841.4762130631


## Testdaten vorbereiten

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    train_data.iloc[:,1:95], train_data.iloc[:,-1], random_state=0, test_size=0.25)

# Create Model Workbench

## Imports

In [13]:
# Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Model selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# Modell
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# CatBoost

https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb

## Erster Versuch

In [14]:
#Model #1 - Catboost

train_pool = Pool(X_train, y_train)
m = CatBoostRegressor(iterations=10000, learning_rate=1, loss_function='MAE', boosting_type='Ordered',task_type='GPU')
m.fit(train_pool, silent=True)
print(m.best_score_)

{'learn': {'MAE': 0.5669221979332273}}


### Feature Importances CatBoost

In [ ]:
feature_importances = m.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

## Zweiter Versuch

In [16]:
#Model #2 - Catboost

train_pool = Pool(X_train, y_train)
m = CatBoostRegressor(iterations=50000, learning_rate=1, loss_function='MAE', boosting_type='Ordered',task_type='GPU')
m.fit(train_pool, silent=True)
print(m.best_score_)

{'learn': {'MAE': 0.28749238208797034}}


### Feature Importances CatBoost

In [17]:
feature_importances = m.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

skew: 4.076301783656566
mean: 3.4809914190580478
rms_kurt: 3.352433021560001
spec_bw_max: 3.1619310461873407
rms_skew: 2.7621800298628836
chroma_stft_5%q: 2.60962868855731
chroma_stft_75%q: 2.5330716263547073
spec_bw_99%q: 2.396200820055166
zero_crossings: 2.394212260630585
chroma_stft_skew: 2.210974835131619
kurt: 2.2019925670217737
chroma_stft_1%q: 2.2019702734849917
spec_cent_max: 2.0454893060040207
chroma_stft_kurt: 2.044862258311427
min: 2.0299909562342555
spec_bw_95%q: 1.9934165482193889
chroma_stft_std: 1.969802539830376
chroma_stft_25%q: 1.8612862283040155
max: 1.8392417760292923
spec_cent_99%q: 1.7512604147211441
chroma_stft_min: 1.7457941287271987
chroma_stft_mean: 1.7070988543795973
rolloff_max: 1.6567401325818834
rms_std: 1.6487141060856076
spec_bw_75%q: 1.589703560765567
absMax: 1.5642231742914035
spec_bw_kurt: 1.5585406214115693
rolloff_99%q: 1.5117624264003107
rolloff_min: 1.5035207109910542
spec_bw_skew: 1.4729828536009388
spec_cent_75%q: 1.4716846830093138
rolloff_kurt

### Feature Importances CatBoost


In [15]:
feature_importances = m.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

skew: 4.650670943408212
mean: 4.044511211246174
min: 3.8143070447859793
max: 3.506119528936808
zero_crossings: 3.021236760917733
absMax: 3.01573497383846
spec_cent_75%q: 2.5692290948180094
kurt: 2.4963686329476102
spec_bw_max: 2.465606317261001
chroma_stft_std: 2.2598725295629527
chroma_stft_75%q: 2.239163425125106
chroma_stft_skew: 2.1650634781557363
rms_kurt: 2.049620994387276
rolloff_min: 2.0380305502193727
spec_bw_99%q: 1.9972544039004516
spec_cent_min: 1.9941023358200385
spec_bw_min: 1.8723248109760737
chroma_stft_kurt: 1.8507307583890595
spec_cent_max: 1.8160196526232901
chroma_stft_mean: 1.7968146791630495
spec_cent_99%q: 1.7658182153211508
chroma_stft_5%q: 1.7174674865246597
spec_bw_95%q: 1.6660885855414274
chroma_stft_min: 1.6611482001439941
chroma_stft_1%q: 1.623560985419079
chroma_stft_25%q: 1.6197646253341869
rms_skew: 1.6100694559429622
spec_bw_75%q: 1.5990716447081543
spec_cent_95%q: 1.550679173059365
rolloff_max: 1.5455279414089582
rolloff_99%q: 1.3826195974280275
spec_b

## Save Model

In [20]:
timestamp = time.time()
#https://catboost.ai/docs/concepts/python-reference_catboost_save_model.html
m.save_model(cat_boost_model+str(timestamp), 
           format="cbm", 
           export_parameters=None,
           pool=None)

## Load Model

In [ ]:
model = CatBoostClassifier()
model.load_model(cat_boost_path);

# Hyperparameter Optimierung

load model

In [ ]:
parameters = {'depth'         : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1,1],
              'iterations'    : [10000]}

In [ ]:
#https://setscholars.net/2019/02/19/how-to-find-optimal-parameters-for-catboost-using-gridsearchcv-for-regression-in-python/
model = CatBoostRegressor(loss_function='MAE', boosting_type='Ordered',task_type='GPU')
parameters = {'depth'         : [6],
              'learning_rate' : [1],
              'iterations'    : [10000]}
grid = GridSearchCV(estimator=model, param_grid = parameters, scoring='neg_mean_absolute_error',cv = 2)
grid.fit(X_train, y_train)

In [10]:
print('SVR(kernel=rbf')
print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))
print("The best score across ALL searched params: ", grid.best_score_)
print("Test-set: mean absolute error : {:.2f}".format(mean_absolute_error(y_test, grid.predict(X_test))))

SVR(kernel=rbf
Best params:
{'depth': 6, 'iterations': 10000, 'learning_rate': 1}

Best cross-validation score: -2.00
Test-set score: -1.64
The best score across ALL searched params:  -1.9965746342232633
Test-set: mean absolute error : 1.64


In [11]:
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n", grid.best_score_)
    
print("\n The best parameters across ALL searched params:\n", grid.best_params_)


 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 -1.9965746342232633

 The best parameters across ALL searched params:
 {'depth': 6, 'iterations': 10000, 'learning_rate': 1}


# Erkenntnisse

# Model selection
 - Random forrest
 - Other GradientBoost (AdaBoost)

 - Feed Forward NN (with Karas)
 - Reccurent Neuronal Net: LSTM (with Karas)
 - Convolutional NN https://www.kaggle.com/michael422/spectrogram-convolution https://www.kaggle.com/michael422/spectrogram-convolution
 
 
 - CatBoost vs. Light GBM vs. XGBoost https://www.kdnuggets.com/2018/03/catboost-vs-light-gbm-vs-xgboost.html
 - CatBoost-GridSearch https://setscholars.net/2019/02/19/how-to-find-optimal-parameters-for-catboost-using-gridsearchcv-for-regression-in-python/